In [2]:
%load_ext kedro.ipython

[07/21/24 01:42:08] INFO     Registered line magic '%reload_kedro'                                   __init__.py:58

                    INFO     Registered line magic '%load_node'                                      __init__.py:60

                    INFO     Resolved project path as: C:\DE                                        __init__.py:171
                             Projects\healthcare-data-processing\healthcare-data-processing.                       
                             To set a different path, run '%reload_kedro <project_root>'                           

As an open-source project, we collect usage analytics. 
We cannot see nor store information contained in a Kedro project. 
You can find out more by reading our privacy notice: 
https://github.com/kedro-org/kedro-plugins/tree/main/kedro-telemetry#privacy-notice 
Do you opt into usage analytics?  [y/N]:

  n


You have opted out of product usage analytics, so none will be collected.


[07/21/24 01:42:24] INFO     Kedro project healthcare-data-processing                               __init__.py:141

                    INFO     Defined global variable 'context', 'session', 'catalog' and            __init__.py:142
                             'pipelines'                                                                           

[07/21/24 01:42:26] INFO     Registered line magic 'run_viz'                                        __init__.py:148

In [26]:
# Load all datasets into dfs

patients_df = catalog.load("patients")
patient_gender_df = catalog.load("patient_gender")
medications_df = catalog.load("medications")
symptoms_df = catalog.load("symptoms")
conditions_df = catalog.load("conditions")
encounters_df = catalog.load("encounters")


[07/21/24 02:53:26] INFO     Loading data from patients (CSVDataset)...                         data_catalog.py:508

[07/21/24 02:53:27] INFO     Loading data from patient_gender (CSVDataset)...                   data_catalog.py:508

                    INFO     Loading data from medications (CSVDataset)...                      data_catalog.py:508

                    INFO     Loading data from symptoms (CSVDataset)...                         data_catalog.py:508

                    INFO     Loading data from conditions (ExcelDataset)...                     data_catalog.py:508

[07/21/24 02:53:29] INFO     Loading data from encounters (ParquetDataset)...                   data_catalog.py:508

In [8]:
# Patients Data Issues

# 1. Add Gender as column for each patient
# 2. Remove Integers from first, last and maiden 

# patients_df.set_index('PATIENT_ID', inplace=True)
# patient_gender_df.set_index('Id', inplace=True)
import pandas as pd

result = pd.merge(patients_df, patient_gender_df, left_on='PATIENT_ID', right_on='Id', how='inner')  # Options: 'inner', 'outer', 'left', 'right'

result.drop(columns=['GENDER_x'], inplace=True)
result.drop(columns=['Id'], inplace=True)
result.rename(columns={'GENDER_y': 'GENDER'}, inplace=True)

result['FIRST'] = result['FIRST'].str.replace(r'\d+$', '', regex=True)
result['LAST'] = result['LAST'].str.replace(r'\d+$', '', regex=True)
result['MAIDEN'] = result['MAIDEN'].str.replace(r'\d+$', '', regex=True)

result.rename(columns={"FIRST": "FIRST_NAME"}, inplace=True)
result.rename(columns={"LAST": "LAST_NAME"}, inplace=True)

display(result)


,PATIENT_ID,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST_NAME,LAST_NAME,SUFFIX,...,STATE,COUNTY,FIPS,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE,INCOME,GENDER
0,54f1059e-6250-3949-6dd0-1dda9b85d22a,2002-01-21,NaN,999-28-3364,S99987398,X44428214X,Ms.,Fredricka,Crist,NaN,...,Texas,Tarrant County,48439.0,76104,0.329154,-97.399553,9990.08,0.00,35460,F
1,92675303-ca5b-136a-169b-e764c5753f06,1997-04-16,NaN,999-62-9859,S99999770,X75016560X,Mr.,Lorenzo,Urrutia,NaN,...,Texas,Tarrant County,48439.0,76006,0.328145,-97.068885,10936.80,0.00,88407,M
2,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,1993-02-04,NaN,999-53-6488,S99979170,X60079936X,Mr.,Luther,MacGyver,NaN,...,Texas,Harris County,48339.0,77357,0.300834,-95.649706,10662.16,0.00,41800,M
3,abc59f62-dc5a-5095-1141-80b4ee8be73b,1995-05-23,NaN,999-37-1058,S99981031,X14759314X,Mrs.,Jacque,Jones,NaN,...,Texas,McLennan County,48309.0,76655,0.315173,-97.292558,11455.93,0.00,41915,F
4,28d7b56c-6056-d0a2-2991-39d6e917216c,1993-11-13,NaN,999-31-6091,S99958903,X63033472X,Mrs.,Angela,Stanton,NaN,...,Texas,Hockley County,48219.0,79336,0.337037,-102.361829,12449.11,0.00,67198,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,3103272c-7646-2160-6e91-5d3eaa82e869,1959-03-31,NaN,999-50-5928,S99988690,X8699556X,Mrs.,Deann,Wunsch,NaN,...,Texas,Galveston County,48201.0,77598,0.295824,-95.176886,14358.74,1230.24,34624,F
9996,34f0ea6d-5e68-4572-4f02-70abf4de4076,1963-12-17,NaN,999-15-2894,S99980164,X53365291X,Mr.,Bo,Jaskolski,NaN,...,Texas,Fort Bend County,NaN,0,0.297249,-95.772813,18605.74,0.00,183359,M
9997,de57c9ca-b881-3560-0f48-678edd01c193,1985-01-01,NaN,999-21-6826,S99986018,X32133517X,Mrs.,Juana,Gil,NaN,...,Texas,El Paso County,48141.0,79927,0.317819,-106.388329,10632.01,0.00,56370,F
9998,9f508536-70b2-d71a-81eb-24b3141ec28a,1992-04-09,NaN,999-56-2350,S99945730,X76311302X,Mr.,Elisha,Price,NaN,...,Texas,McLennan County,48099.0,76561,0.314505,-97.390693,11431.06,0.00,66124,M


In [12]:
# Conditions Table Data Issues
# 1. Patient should be lower case


conditions_df['PATIENT'] = conditions_df['PATIENT'].str.lower()

display(conditions_df)


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2020-04-21,NaN,28d7b56c-6056-d0a2-2991-39d6e917216c,402dfaeb-1888-f562-4651-f75203eb1e28,200936003,Lupus erythematosus
1,2001-11-07,NaN,239ae86a-96db-6211-9042-d3f2850aabb8,10d39d6f-7d98-8d03-8866-0a7cd363890b,200936003,Lupus erythematosus
2,2001-11-07,NaN,239ae86a-96db-6211-9042-d3f2850aabb8,10d39d6f-7d98-8d03-8866-0a7cd363890b,271737000,Anemia (disorder)
3,2017-11-05,NaN,6c434506-fb4b-3e3f-c19d-553dec3b6c17,add4fa37-cabe-a5aa-f9da-8117464bad04,200936003,Lupus erythematosus
4,1989-09-15,NaN,6872def5-772f-427c-3053-de6e1c71ce0a,d8a684cb-6ce1-5909-59e7-3422469d18d0,200936003,Lupus erythematosus
...,...,...,...,...,...,...
4144,2016-01-04,NaN,8b509749-aaab-30b8-9a67-e8620eec19f0,3ee65f54-bcba-57db-6132-d130a9e5ef90,271737000,Anemia (disorder)
4145,2010-06-22,NaN,34f0ea6d-5e68-4572-4f02-70abf4de4076,404c6d33-70e4-0d6e-f6f9-1202ed8efdf4,200936003,LUPUS ERYTHEMATOSUS
4146,2010-06-22,NaN,34f0ea6d-5e68-4572-4f02-70abf4de4076,404c6d33-70e4-0d6e-f6f9-1202ed8efdf4,271737000,Anemia (disorder)
4147,1996-12-15,NaN,8d116ace-7fb6-7d5f-b86b-94547cbd796d,59214785-c4a8-8e7f-f583-10eb3ee824e6,200936003,Lupus erythematosus


In [36]:
# Medications Table Data Issues
# 1. Encounter lower case
# 2. Reason Code Integer


medications_df['ENCOUNTER'] = medications_df['ENCOUNTER'].str.lower()
medications_df['REASONCODE'] = medications_df['REASONCODE'].astype(str).str.replace('.0', '', regex=False)

display(medications_df)

,START,STOP,PATIENT,PAYER,ENCOUNTER,CODE,DESCRIPTION,BASE_COST,PAYER_COVERAGE,DISPENSES,TOTALCOST,REASONCODE,REASONDESCRIPTION
0,2020-04-22T02:47:09Z,NaN,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402dfaeb-1888-f562-4651-f75203eb1e28,849574,Naproxen sodium 220 MG Oral Tablet,274.28,0.0,45,12342.60,200936003,Lupus erythematosus
1,2020-04-22T02:47:09Z,2020-05-17T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,402dfaeb-1888-f562-4651-f75203eb1e28,312615,predniSONE 20 MG Oral Tablet,16.04,0.0,1,16.04,200936003,Lupus erythematosus
2,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,fbc2f3b3-ab93-707e-ad9e-337990345f32,241834,cycloSPORINE modified 100 MG Oral Capsule,473.61,0.0,1,473.61,200936003,Lupus erythematosus
3,2021-08-01T02:47:09Z,2021-08-15T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,fbc2f3b3-ab93-707e-ad9e-337990345f32,312615,predniSONE 20 MG Oral Tablet,8.21,0.0,1,8.21,200936003,Lupus erythematosus
4,2023-05-18T02:47:09Z,2023-06-14T02:47:09Z,28d7b56c-6056-d0a2-2991-39d6e917216c,0133f751-9229-3cfd-815f-b6d4979bdd6a,e3989f64-4d5b-32e0-9edc-7874a02f45e8,241834,cycloSPORINE modified 100 MG Oral Capsule,408.04,0.0,1,408.04,200936003,Lupus erythematosus
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22232,2019-12-28T14:26:30Z,2020-01-17T14:26:30Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,49e9369b-ef5e-22cb-5f7c-b5e3035841d2,312615,predniSONE 20 MG Oral Tablet,12.24,0.0,1,12.24,200936003,Lupus erythematosus
22233,2021-09-22T14:26:30Z,2021-10-14T14:26:30Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,734afbd6-4794-363b-9bc0-6a3981533ed5,da11a1e4-491d-50ac-2704-c80862ef10d9,241834,cyclosporine modified 100 mg oral capsule,265.17,0.0,1,265.17,200936003,Lupus erythematosus
22234,2021-09-22T14:26:30Z,2021-10-14T14:26:30Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,734afbd6-4794-363b-9bc0-6a3981533ed5,da11a1e4-491d-50ac-2704-c80862ef10d9,312615,predniSONE 20 MG Oral Tablet,13.37,0.0,1,13.37,200936003,Lupus erythematosus
22235,2023-04-13T13:10:12Z,2023-05-10T13:10:12Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,734afbd6-4794-363b-9bc0-6a3981533ed5,48bd83c4-5f36-4bd4-a8ac-1e0a5d1afc2d,241834,cycloSPORINE modified 100 MG Oral Capsule,233.49,0.0,1,233.49,200936003,Lupus erythematosus


In [30]:
# Encounters Table Data Issues
# 1. Patient should be in lower case
# 2. Reason Code remove decimals


encounters_df['PATIENT'] = encounters_df['PATIENT'].str.lower()
encounters_df['REASONCODE'] = encounters_df['REASONCODE'].astype(str).str.replace('.0', '', regex=False).str.replace('nan', '', regex=False)
display(encounters_df)

,Id,START,STOP,PATIENT,ORGANIZATION,PROVIDER,PAYER,ENCOUNTERCLASS,CODE,DESCRIPTION,BASE_ENCOUNTER_COST,TOTAL_CLAIM_COST,PAYER_COVERAGE,REASONCODE,REASONDESCRIPTION
0,6539c675-dcf2-cace-a90d-7f097d1799b3,2014-02-10T11:00:08Z,2014-02-10T11:15:08Z,54f1059e-6250-3949-6dd0-1dda9b85d22a,ef42ec52-a1c1-3fc5-8e52-da46f7f3970c,35088bec-f756-35d8-8140-b2b017c0eacc,8fa6c185-e44e-3e34-8bd8-39be8694f4ce,wellness,410620009,Well child visit (procedure),133.36,405.36,0.0,,None
1,748a443c-9819-4f3a-8648-0096ace6d3fd,2014-06-04T05:19:43Z,2014-06-04T05:34:43Z,92675303-ca5b-136a-169b-e764c5753f06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,410620009,Well child visit (procedure),133.36,269.36,0.0,,None
2,678e03b1-21ac-fbfa-3506-17bee72a6335,2015-06-10T05:19:43Z,2015-06-10T05:34:43Z,92675303-ca5b-136a-169b-e764c5753f06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,,None
3,45987160-8815-858e-893b-6490a63e919c,2015-04-10T03:27:04Z,2015-04-10T03:42:04Z,a0b63e97-b6fd-5fe1-8f2d-2bec915efa97,44f0ad91-f33b-3fe5-ba4f-868503d593fe,a79c71d6-e0b4-3450-a0e1-ee08146aeb74,734afbd6-4794-363b-9bc0-6a3981533ed5,wellness,162673000,General examination of patient (procedure),133.36,677.36,0.0,,None
4,c56e3b19-5a51-0fc8-a82e-8bf89575ca71,2016-06-15T05:19:43Z,2016-06-15T05:34:43Z,92675303-ca5b-136a-169b-e764c5753f06,02078b23-a943-3175-b5f0-8d386dd8eb56,af22b73b-d38f-3d92-8a95-bd57ac2dc3f1,b046940f-1664-3047-bca7-dfa76be352a4,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104257,da11a1e4-491d-50ac-2704-c80862ef10d9,2021-09-22T14:26:30Z,2021-09-22T14:41:30Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,2ed3f13d-06a1-3d43-a93f-dfa249383215,83840821-b054-3e4e-9460-698abf776955,734afbd6-4794-363b-9bc0-6a3981533ed5,virtual,185347001,Encounter for problem,75.00,75.00,0.0,200936003,Lupus erythematosus
104258,88c2024b-d1c2-71fa-8e54-b96dc5acf4cb,2021-09-30T13:10:12Z,2021-09-30T13:25:12Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,2ed3f13d-06a1-3d43-a93f-dfa249383215,83840821-b054-3e4e-9460-698abf776955,734afbd6-4794-363b-9bc0-6a3981533ed5,outpatient,33879002,Administration of vaccine to produce active im...,113.77,249.77,0.0,,None
104259,cb796012-5470-f89d-dfe4-388bc039e945,2022-03-31T13:10:12Z,2022-03-31T13:25:12Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,2fffe368-17d0-35fe-8335-1561d92b4077,79f6af3a-6baf-3d0f-b745-ae31827ec70e,734afbd6-4794-363b-9bc0-6a3981533ed5,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,,None
104260,7f1af5b9-da14-975c-cb13-59cbd7bdb447,2023-04-06T13:10:12Z,2023-04-06T13:25:12Z,8d116ace-7fb6-7d5f-b86b-94547cbd796d,2fffe368-17d0-35fe-8335-1561d92b4077,79f6af3a-6baf-3d0f-b745-ae31827ec70e,734afbd6-4794-363b-9bc0-6a3981533ed5,wellness,162673000,General examination of patient (procedure),133.36,269.36,0.0,,None


In [34]:
# Symptoms Table Data Issues
# 1. Add Gender

result = pd.merge(
        symptoms_df, patient_gender_df, left_on="PATIENT", right_on="Id", how="inner"
    )

# TODO add rename and drop utility functions
result.drop(columns=["GENDER_x"], inplace=True)
result.drop(columns=["Id"], inplace=True)
result.rename(columns={"GENDER_y": "GENDER"}, inplace=True)

# result["FIRST"] = result["FIRST"].str.replace(r"\d+$", "", regex=True)
# result["LAST"] = result["LAST"].str.replace(r"\d+$", "", regex=True)
# result["MAIDEN"] = result["MAIDEN"].str.replace(r"\d+$", "", regex=True)

# result.rename(columns={"FIRST": "FIRST_NAME"}, inplace=True)
# result.rename(columns={"LAST": "LAST_NAME"}, inplace=True)

display(result) 

,PATIENT,RACE,ETHNICITY,AGE_BEGIN,AGE_END,PATHOLOGY,NUM_SYMPTOMS,SYMPTOMS,GENDER
0,28d7b56c-6056-d0a2-2991-39d6e917216c,white,hispanic,26,NaN,Lupus erythematosus,4,Rash:34;Joint Pain:39;Fatigue:9;Fever:12,F
1,6c434506-fb4b-3e3f-c19d-553dec3b6c17,black,hispanic,58,NaN,Lupus erythematosus,4,Rash:19;Joint Pain:44;Fatigue:48;Fever:15,F
2,44a8ca45-6c6e-38bb-fac0-ddbf7a7ee3a4,white,nonhispanic,50,NaN,Lupus erythematosus,4,Rash:2;Joint Pain:32;Fatigue:12;Fever:6,M
3,780ec78c-22a0-fcdb-17c6-ae9b2fcace9c,white,hispanic,39,NaN,Lupus erythematosus,4,Rash:30;Joint Pain:30;Fatigue:41;Fever:19,F
4,cf5956bb-34f2-841b-2505-57b99991c377,white,nonhispanic,29,NaN,Lupus erythematosus,3,Rash:28;Joint Pain:26;Fatigue:33;Fever:5,F
...,...,...,...,...,...,...,...,...,...
1418,110d2862-432d-5b0d-c83c-0096fc84fc64,native,hispanic,31,NaN,Lupus erythematosus,4,Rash:34;Joint Pain:41;Fatigue:41;Fever:19,F
1419,110d2862-432d-5b0d-c83c-0096fc84fc64,native,hispanic,31,NaN,Anemia (disorder),4,Rash:34;Joint Pain:41;Fatigue:41;Fever:19,F
1420,ae46ea17-eb0a-6159-14dd-9d9fa7956be4,white,hispanic,57,NaN,Lupus erythematosus,4,Rash:10;Joint Pain:33;Fatigue:44;Fever:12,F
1421,8b509749-aaab-30b8-9a67-e8620eec19f0,white,hispanic,39,NaN,Lupus erythematosus,4,Rash:17;Joint Pain:8;Fatigue:7;Fever:16,M
